# Running a CAT based on a synthetic correct/incorrect 3PL IRT model

This example shows how to run a CAT based on a synthetic correct/incorrect 3PL
IRT model.

Import order is important. We put ComputerAdaptiveTesting last so we get the
extra dependencies.

In [1]:
using Makie
import Pkg
import Random
using Distributions: Normal, cdf
using AlgebraOfGraphics
using ComputerAdaptiveTesting
using ComputerAdaptiveTesting.ExtraDistributions: NormalScaledLogistic
using ComputerAdaptiveTesting.Sim: auto_responder
using ComputerAdaptiveTesting.NextItemRules: AbilityVarianceStateCriterion
using ComputerAdaptiveTesting.TerminationConditions: FixedItemsTerminationCondition
using ComputerAdaptiveTesting.Aggregators: PriorAbilityEstimator, MeanAbilityEstimator, LikelihoodAbilityEstimator
using ComputerAdaptiveTesting.ItemBanks
using ComputerAdaptiveTesting.Integrators
using ComputerAdaptiveTesting.Responses: BooleanResponse
import ComputerAdaptiveTesting.IntegralCoeffs
using CATPlots

@automakie()

Now we are read to generate our synthetic data using the supplied DummyData
module. We generate an item bank with 100 items and fake responses for 3
testees.

In [2]:
using ComputerAdaptiveTesting.DummyData: dummy_full, std_normal, SimpleItemBankSpec, StdModel3PL
using ComputerAdaptiveTesting.MathTraits
(item_bank, question_labels, abilities, responses) = dummy_full(
    Random.default_rng(42),
    SimpleItemBankSpec(StdModel3PL(), OneDimContinuousDomain(), BooleanResponse());
    num_questions=100,
    num_testees=3
)

(ComputerAdaptiveTesting.ItemBanks.GuessItemBank{ComputerAdaptiveTesting.ItemBanks.TransferItemBank{Distributions.Logistic{Float64}}}([0.0, 0.237252836038323, 0.0, 0.03275106529832784, 0.0, 0.0351587483937106, 0.0, 0.0, 0.04056768006220382, 0.21310502748660284  …  0.06738068183406366, 0.19426590106127, 0.0, 0.0, 0.0, 0.18834775314017285, 0.4221605931572538, 0.02555633445480085, 0.24631329019983972, 0.0], ComputerAdaptiveTesting.ItemBanks.TransferItemBank{Distributions.Logistic{Float64}}(Distributions.Logistic{Float64}(μ=0.0, θ=0.5875440658049353), [0.6804008389677381, -0.15011378722397994, 0.3078552041137613, 1.2152655391020046, -0.6736898408785245, 0.5281708444946384, -0.874962849768716, 0.008528062588338724, -0.7162933877053225, -0.02861762730735681  …  0.0906679224857289, -1.1164722667382294, 0.8201101250484729, 1.4538806126010566, -0.02739403699450549, 1.6254265024166588, 0.9589275552361124, -0.03637480223512257, 1.6125870056275518, -0.27419997700490384], [1.188322429561845, 0.6367

Simulate a CAT for each testee and record it using CatRecorder.
CatRecorder collects information which can be used to draw different types of plots.

In [3]:
max_questions = 99
integrator = FixedGKIntegrator(-6, 6, 80)
dist_ability_est = PriorAbilityEstimator(std_normal)
ability_estimator = MeanAbilityEstimator(dist_ability_est, integrator)
rules = CatRules(
    ability_estimator,
    AbilityVarianceStateCriterion(dist_ability_est, integrator),
    FixedItemsTerminationCondition(max_questions)
)

points = 500
xs = range(-2.5, 2.5, length=points)
raw_estimator = LikelihoodAbilityEstimator()
recorder = CatRecorder(xs, responses, integrator, raw_estimator, ability_estimator)
for testee_idx in axes(responses, 2)
    tracked_responses, θ = run_cat(
        CatLoopConfig(
            rules=rules,
            get_response=auto_responder(@view responses[:, testee_idx]),
            new_response_callback=(tracked_responses, terminating) -> recorder(tracked_responses, testee_idx, terminating),
        ),
        item_bank
    )
    true_θ = abilities[testee_idx]
    abs_err = abs(θ - true_θ)
end

Make a plot showing how the estimated value evolves during the CAT.
We also plot the 'true' values used to generate the responses.

In [4]:
conv_lines_fig = ability_evolution_lines(recorder; abilities=abilities)
conv_lines_fig

FigureGrid()

Make an interactive plot, showing how the distribution of the ability
likelihood evolves.

In [5]:
conv_dist_fig = lh_evolution_interactive(recorder; abilities=abilities)
conv_dist_fig

Figure()

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*